# 概要
* 複数インデックスのマージにいよいよトライする

# 参考
* そもそもtree構造が前提なのでtreeのindexを確認する
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html
* https://note.com/npaka/n/n5d4ff4380055

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [22]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

documents = SimpleDirectoryReader('story/dai').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
d_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
#index = GPTTreeIndex(documents, prompt_helper=prompt_helper)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 序章（デルパ! イルイル! / ダイ爆発!!!）
魔王ハドラー率いる魔王軍と勇者率いる勇者パー...
DEBUG:root:> Adding chunk: そこに勇者の家庭教師を名乗る謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホ...
DEBUG:root:> Adding chunk: アバンは弟子たちを護るために自己犠牲呪文・メガンテを使い、その命を散らすもハドラーは生き残り、...
DEBUG:root:> Adding chunk: アバンへの誤解から魔王軍に加わっていたヒュンケルもまた改心し、ダイ達の仲間となる。一行は氷炎将...
DEBUG:root:> Adding chunk: 人間に迫害されて家族を引き裂かれた過去を持つバランはダイに対し、魔王軍に加わり自分と共に人間を...
DEBUG:root:> Adding chunk: ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリ...
DEBUG:root:> Adding chunk: ダイとバランは魔宮の門を破壊した後、ハドラーと対峙するが、バランはハドラーの体内に魔界の超強力...
DEBUG:root:> Adding chunk: 一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒に...
DEBUG:root:> Adding chunk: バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある...
DEBUG:root:> Adding chunk: 

エピローグ
地上に帰還したダイが仲間たちに迎えられ、皆が勝利を喜びあう中、バーンパレスの戦...
DEBUG:root:> Adding chunk: ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (4): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=237 request_id=08b73722751d799b983db0ea5244cb9e response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u5e8f\\u7ae0\\uff08\\u30c7\\u30eb\\u30d1! \\u30a4\\u30eb\\u30a4\\u30eb! / \\u30c0\\u30a4\\u7206\\u767a!!!\\uff09 \\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\\u52c7\\u8005\\u7387\\u3044\\u308b\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u6226\\u3044\\u306f\\u3001\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u3067\\u7d42\\u308f\\u308a\\u300

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 91164
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=144 request_id=df6d45cb30d4b6df85b5651a128cc5ec response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4404 tokens


In [23]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
g_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
#index = GPTTreeIndex(documents, prompt_helper=prompt_helper)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk: 

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8...
DEBUG:root:> Adding chunk: 悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き...
DEBUG:root:> Adding chunk: 

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過...
DEBUG:root:> Adding chunk: 仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と...
DEBUG:root:> Adding chunk: フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落と...
DEBUG:root:> Adding chunk: トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空に...
DEBUG:root:> Adding chunk: 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強...
DEBUG:root:> Adding chunk: 天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人...
DEBUG:root:> Adding chunk: ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None da

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=140 request_id=509168ba887440db296dd8f059999b8d response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4371 tokens


In [24]:
d_index.set_text("ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊したが、バーンパレスへの立ち入りは出来ずにいた。そこへバーンの直属の暗殺者である死神キルバーンが襲撃し、バーンの真の目的を知ったバランもバーンパレスを訪れ、ダイ一行と共闘することになる。ダイ一行はバーンパレスを破壊し、バーンとの直接対決において敗北したが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、ダイ一行はカール王国女王のフローラの協力を得て、大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還したアバンであった。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。ダイはバーンを倒し、神の涙によって世界中の人々が協力し、爆発を阻止する。その後、ダイ一行は地上に帰還し、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルであり、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであった。最終的にダイはバーンを倒し、世界を救うことに成功した")

In [25]:
g_index.set_text("孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をし、ウーロンやヤムチャらと出会いながら、世界征服を企むピラフ一味を阻止する。孫悟空は亀仙人の下で修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会では、鶴仙流の天津飯と激突し準優勝を飾る。その後、悟空は牛魔王の娘・チチと結婚し、天下一武道会でマジュニアと決勝戦で激突し初の天下一武道会優勝を飾る。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。フリーザと悟空の戦いから始まり、人造人間・セル編、魔人ブウ編と続き、セルゲームを開催したセルを撃破するため、悟空は自分の命を落とすことを選択します。その後、悟空の息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破します。そして、7年後には魔人ブウが復活し、悟空とベジータが界王神界で最終決戦を行い、ドラゴンボールの協力を得て魔人ブウを消滅させます。10年後、悟空は孫のパンと共に天下一武道会に出場し、魔人ブウの生まれ変わりである少年・ウーブと出会い、修行に旅立ち、物語は幕を閉じます。")

In [39]:
g_index.query("魔人ブウを倒したのは誰ですか？",similarity_top_k=2)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u9b54\\u4eba\\u30d6\\u30a6\\u3092\\u5012\\u3057\\u305f\\u306e\\u306f\\u8ab0\\u3067\\u3059\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=18 request_id=62ac1f3b30eac6969fba69e9972fbe3d response_code=200
DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node c28d2f1f-a7b6-4b60-bad7-c343ea6e8f43] [Similarity score:             0.839056] 天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空...
> [Node e28914e6-173b-49f8-9a

Response(response='\n孫悟天と少年トランクスがフュージョンして誕生した戦士・ゴテンクスが、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウを完全に消滅させました。', source_nodes=[SourceNode(source_text='天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。\n\n一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、戦士たちを次々と吸収し姿を変えていく魔人ブウに苦戦を強いられる。危機に陥った悟飯らを救うため現世に舞い戻った悟空とベジータは、界王神界で真の姿となった魔人ブウとの最終決戦に臨む', doc_id='e968a6f3-c33b-4197-9fae-1c1dacf31249', extra_info=None, node_info={'start': 2573, 'end': 2911}, similarity=0.8390556653121682), SourceNode(source_text='ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。\n\nそれから10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。悟空はウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じる。\n\n', doc_id='e968a6f3-c33b-4197-9fae-1c1dacf31249', extra_info=None, node_info={'start': 2912, 'end': 3083}, similarity=0.8384767195385219)], extra_info=No

In [40]:
g_index.query("魔人ブウの生まれ変わりは誰ですか？",similarity_top_k=2)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u9b54\\u4eba\\u30d6\\u30a6\\u306e\\u751f\\u307e\\u308c\\u5909\\u308f\\u308a\\u306f\\u8ab0\\u3067\\u3059\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=26 request_id=fcdf4a7f37349222d944c017398cf9fd response_code=200
DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node e28914e6-173b-49f8-9a7c-5ebc187c0c1c] [Similarity score:             0.858575] ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。

それから10年後、悟空は孫のパンと共に天下一武道会...
> [Node c28d2f1

Response(response='\n魔人ブウの生まれ変わりは少年・ウーブであり、悟空とベジータが界王神界で最終決戦に臨むために現世に舞い戻った際に登場しました。', source_nodes=[SourceNode(source_text='ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。\n\nそれから10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。悟空はウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じる。\n\n', doc_id='e968a6f3-c33b-4197-9fae-1c1dacf31249', extra_info=None, node_info={'start': 2912, 'end': 3083}, similarity=0.8585748384156198), SourceNode(source_text='天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。\n\n一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、戦士たちを次々と吸収し姿を変えていく魔人ブウに苦戦を強いられる。危機に陥った悟飯らを救うため現世に舞い戻った悟空とベジータは、界王神界で真の姿となった魔人ブウとの最終決戦に臨む', doc_id='e968a6f3-c33b-4197-9fae-1c1dacf31249', extra_info=None, node_info={'start': 2573, 'end': 2911}, similarity=0.8472688410804206)], extra_info=None)

In [53]:
#keyword_table = GPTSimpleKeywordTableIndex([d_index, g_index], max_keywords_per_chunk=10)
#keyword_table = GPTSimpleKeywordTableIndex([d_index], max_keywords_per_chunk=10)
index = GPTListIndex([d_index, g_index])

DEBUG:root:> Adding chunk: ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊した...
DEBUG:root:> Adding chunk: 孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をし、ウーロンやヤムチャらと出会いな...
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens


In [49]:
# OK
index.query("魔人ブウの生まれ変わりは誰ですか？")

DEBUG:root:> Searching in chunk: ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊した...
DEBUG:root:> Searching in chunk: 孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をし、ウーロンやヤムチャらと出会いな...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\u30c0\\u30a4\\u4e00\\u884c\\u306f\\u3001\\u5927\\u9b54\\u738b\\u30d0\\u30fc\\u30f3\\u3068\\u306e\\u5bfe\\u5cd9\\u306e\\u305f\\u3081\\u306b\\u4e16\\u754c\\u5404\\u56fd\\u306e\\u738b\\u304b\\u3089\\u5354\\u529b\\u3092\\u5f97\\u3066\\u6226\\u529b\\u3092\\u84c4\\u3048\\u3001\\u9b3c\\u5ca9\\u57ce\\u3092\\u7834\\u58ca\\u3057\\u305f\\u304c\\u3001\\u30d0\\u30fc\\u30f3\\u30d1\\u30ec\\u30b9\\u3078\\u306e\\u7acb\\u3061\\u5165\\u308a\\u306f\\u51fa\\u6765\\u305a\\u306b\\u3044\\u305f\\u3002\\u305d\\u3053\\u3078\\u30d0\\u30fc\\u30f3\\u306e\\u76f4\\u5c5e\\u306e\\u6697\\u6bba\\u8005\\u3067\\u3042\\u308b\\u6b7b\\u795e\\u30ad\\u30eb\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2822 request_id=940277cfb898e5d89c6c27746785cfff response_code=200
DEBUG:root:
魔人ブウの生まれ変わりは、孫悟空とベジータが界王神界で最終決戦を行った10年後に出会った少年・ウーブです。
DEBUG:root:> Refined response: 
魔人ブウの生まれ変わりは、孫悟空とベジータが界王神界で最終決戦を行った10年後に出会った少年・ウーブです。
INFO:root:> [query] Total LLM token usage: 1977 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\n魔人ブウの生まれ変わりは、孫悟空とベジータが界王神界で最終決戦を行った10年後に出会った少年・ウーブです。', source_nodes=[SourceNode(source_text='ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊したが、バーンパレスへの立ち入りは出来ずにいた。そこへバーンの直属の暗殺者である死神キルバーンが襲撃し、バーンの真の目的を知ったバランもバーンパレスを訪れ、ダイ一行と共闘することになる。ダイ一行はバーンパレスを破壊し、バーンとの直接対決において敗北したが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、ダイ一行はカール王国女王のフローラの協力を得て、大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還したアバンであった。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。ダイはバーンを倒し、神の涙によって世界中の人々が協力し、爆発を阻止する。その後、ダイ一行は地上に帰還し、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルであり、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであった。最終的にダイはバーンを倒し、世界を救うことに成功した', doc_id='00fef2be-1b54-41ae-b692-0b581e2e3e5a', extra_info=None, node_info={'start': 0, 'end': 659}, similarity=None), SourceNode(source_text='孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をし、ウーロンやヤムチャらと出会いながら、世界征服を企むピラフ一味を阻止する。孫悟空は亀仙人の下で修行を

In [50]:
# NG
index.query("キルバーンの主は誰ですか？")

DEBUG:root:> Searching in chunk: ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊した...
DEBUG:root:> Searching in chunk: 孫悟空少年編では、孫悟空とブルマがドラゴンボールを探して旅をし、ウーロンやヤムチャらと出会いな...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\u30c0\\u30a4\\u4e00\\u884c\\u306f\\u3001\\u5927\\u9b54\\u738b\\u30d0\\u30fc\\u30f3\\u3068\\u306e\\u5bfe\\u5cd9\\u306e\\u305f\\u3081\\u306b\\u4e16\\u754c\\u5404\\u56fd\\u306e\\u738b\\u304b\\u3089\\u5354\\u529b\\u3092\\u5f97\\u3066\\u6226\\u529b\\u3092\\u84c4\\u3048\\u3001\\u9b3c\\u5ca9\\u57ce\\u3092\\u7834\\u58ca\\u3057\\u305f\\u304c\\u3001\\u30d0\\u30fc\\u30f3\\u30d1\\u30ec\\u30b9\\u3078\\u306e\\u7acb\\u3061\\u5165\\u308a\\u306f\\u51fa\\u6765\\u305a\\u306b\\u3044\\u305f\\u3002\\u305d\\u3053\\u3078\\u30d0\\u30fc\\u30f3\\u306e\\u76f4\\u5c5e\\u306e\\u6697\\u6bba\\u8005\\u3067\\u3042\\u308b\\u6b7b\\u795e\\u30ad\\u30eb\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=5041 request_id=906a092db4e7004cf8ad79590cf1c988 response_code=200
DEBUG:root:
キルバーンの主は、魔界におけるバーンのライバルであり、孫悟空とベジータが界王神界で最終決戦を行い、ドラゴンボールの協力を得て魔人ブウを消滅させた後、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーです。
DEBUG:root:> Refined response: 
キルバーンの主は、魔界におけるバーンのライバルであり、孫悟空とベジータが界王神界で最終決戦を行い、ドラゴンボールの協力を得て魔人ブウを消滅させた後、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーです。
INFO:root:> [query] Total LLM token usage: 2169 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\nキルバーンの主は、魔界におけるバーンのライバルであり、孫悟空とベジータが界王神界で最終決戦を行い、ドラゴンボールの協力を得て魔人ブウを消滅させた後、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーです。', source_nodes=[SourceNode(source_text='ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊したが、バーンパレスへの立ち入りは出来ずにいた。そこへバーンの直属の暗殺者である死神キルバーンが襲撃し、バーンの真の目的を知ったバランもバーンパレスを訪れ、ダイ一行と共闘することになる。ダイ一行はバーンパレスを破壊し、バーンとの直接対決において敗北したが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、ダイ一行はカール王国女王のフローラの協力を得て、大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還したアバンであった。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。ダイはバーンを倒し、神の涙によって世界中の人々が協力し、爆発を阻止する。その後、ダイ一行は地上に帰還し、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルであり、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであった。最終的にダイはバーンを倒し、世界を救うことに成功した', doc_id='00fef2be-1b54-41ae-b692-0b581e2e3e5a', extra_info=None, node_info={'start': 0, 'end': 659}, similarity=None), SourceNode(source_text='孫悟空少年編では、孫

In [60]:
# set query config
#query_configsを利用するにはwrapperクラスが必要
query_configs = [
    {
        "index_struct_type": "simple_dict ",
        "query_mode": "default",
        #"query_kwargs": {
        #    "similarity_top_k": 1
        #}
    }
]

In [61]:
graph = ComposableGraph.build_from_index(index)
graph.save_to_disk("merge.json")

# [Optional] load from disk
graph = ComposableGraph.load_from_disk("merge.json")

In [62]:
# NG
graph.query("キルバーンの主は誰ですか？", query_configs=query_configs)

DEBUG:root:> Searching in chunk: ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊した...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u4e3b\\u306f\\u8ab0\\u3067\\u3059\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 429 285
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=15009 request_id=c5f1203fe6fedda1c3f07b29b5e8ca5c response_code=429
INFO:openai:error_code=None error_message='The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.o

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1488 request_id=539e891603029dcae372705a3f018fb5 response_code=200
DEBUG:root:
キルバーンの主はピッコロ大魔王です。
DEBUG:root:> Initial response: 
キルバーンの主はピッコロ大魔王です。
INFO:root:> [query] Total LLM token usage: 562 tokens
INFO:root:> [query] Total embedding token usage: 16 tokens
DEBUG:root:> Refine context: 
キルバーンの主はピッコロ大魔王です。...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u4e3b\\u306f\\u8ab0\\u3067\\u3059\\u304b\\uff1f\\nWe have provided an existing answer: \\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u4e3b\\u306f\\u3001\\u9b54\\u754c\\u306b\\u304a\\u3051\\u308b\\u30d0\\u30fc\\u30f3\\u306e\\u30e9\\u30a4\\u30d0\\u30eb\\u3067\

Response(response='\n\nキルバーンの主は、魔界におけるバーンのライバルで、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであり、ピッコロ大魔王です。', source_nodes=[SourceNode(source_text='ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊したが、バーンパレスへの立ち入りは出来ずにいた。そこへバーンの直属の暗殺者である死神キルバーンが襲撃し、バーンの真の目的を知ったバランもバーンパレスを訪れ、ダイ一行と共闘することになる。ダイ一行はバーンパレスを破壊し、バーンとの直接対決において敗北したが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、ダイ一行はカール王国女王のフローラの協力を得て、大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還したアバンであった。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。ダイはバーンを倒し、神の涙によって世界中の人々が協力し、爆発を阻止する。その後、ダイ一行は地上に帰還し、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルであり、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであった。最終的にダイはバーンを倒し、世界を救うことに成功した', doc_id='00fef2be-1b54-41ae-b692-0b581e2e3e5a', extra_info=None, node_info={'start': 0, 'end': 659}, similarity=None), SourceNode(source_text='\n\nエピローグ\n地上に帰還したダイが仲間たちに迎えられ、皆が勝利を喜びあう中、バーンパ

In [63]:
# NG
graph.query("悟空は魔人ブウをどのように倒しましたか？", query_configs=query_configs)

DEBUG:root:> Searching in chunk: ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊した...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u609f\\u7a7a\\u306f\\u9b54\\u4eba\\u30d6\\u30a6\\u3092\\u3069\\u306e\\u3088\\u3046\\u306b\\u5012\\u3057\\u307e\\u3057\\u305f\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=18 request_id=74fc4bf5148a0853489229adfef49a78 response_code=200
DEBUG:gpt_index.indices.utils:> Top 1 nodes:
> [Node a778b8a0-f24a-4335-bd55-6d7fb923ccb2] [Similarity score:             0.842038] ダイ一行も魔界の名工ロン・

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3610 request_id=018ada09c71fe4939701f7c3138b990d response_code=200
DEBUG:root:
悟空は、界王神界で真の姿となった魔人ブウとの最終決戦で、孫悟天と少年トランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスと共に戦い、最終的に魔人ブウを倒しました。
DEBUG:root:> Initial response: 
悟空は、界王神界で真の姿となった魔人ブウとの最終決戦で、孫悟天と少年トランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスと共に戦い、最終的に魔人ブウを倒しました。
INFO:root:> [query] Total LLM token usage: 664 tokens
INFO:root:> [query] Total embedding token usage: 30 tokens
DEBUG:root:> Refine context: 
悟空は、界王神界で真の姿となった魔人ブウとの最終決戦で、孫悟天と少年トランクスが「フュージョ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u609f\\u7a7a\\u306f\\u9b54\\u4eba\\u30d6\\u30a6\\u3092\\u3069\\u306e\\u3088\\u3046\\u306b\\u5012\\u3057\\u307e\\u3057\\u305f\\u304b\\uf

Response(response='\n\n悟空は、界王神界で真の姿となった魔人ブウとの最終決戦で、孫悟天と少年トランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスと共に戦い、最終的に極限バーストという技を使って魔人ブウを一撃で倒すことができました。極限バーストは、悟空が蓄えた新たな力を使って、魔人ブウを一撃で倒すことができる技です。', source_nodes=[SourceNode(source_text='ダイ一行は、大魔王バーンとの対峙のために世界各国の王から協力を得て戦力を蓄え、鬼岩城を破壊したが、バーンパレスへの立ち入りは出来ずにいた。そこへバーンの直属の暗殺者である死神キルバーンが襲撃し、バーンの真の目的を知ったバランもバーンパレスを訪れ、ダイ一行と共闘することになる。ダイ一行はバーンパレスを破壊し、バーンとの直接対決において敗北したが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、ダイ一行はカール王国女王のフローラの協力を得て、大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還したアバンであった。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。ダイはバーンを倒し、神の涙によって世界中の人々が協力し、爆発を阻止する。その後、ダイ一行は地上に帰還し、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルであり、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであった。最終的にダイはバーンを倒し、世界を救うことに成功した', doc_id='00fef2be-1b54-41ae-b692-0b581e2e3e5a', extra_info=None, node_info={'start': 0, 'end': 659}, similarity=None), Sour